In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

In [2]:
from utils import loader
from utils import debugger 
from utils import preprocessing as pp 
from utils import misc
from utils import constant
import implementations as impl
import cost
import model

In [3]:
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, x, ids = loader.load_csv_data(DATA_TRAIN_PATH)

In [5]:
to_replace = [(constant.UNDEF_VAL, 'mean')]

augment_param_0 = {
    'degrees': [2,3],
    'add_bias' : True,
    'add_cross': True,
    'add_tanh': True,
    'cumulative': False
}

augment_param_1 = {
    'degrees': [2],
    'add_bias' : True,
    'add_cross': True,
    'add_tanh': True,
    'cumulative': True
}

augment_param_23 = {
    'degrees': [2],
    'add_bias' : True,
    'add_cross': True,
    'add_tanh': True,
    'cumulative': True
}

augment_param_list = [augment_param_0, augment_param_1, augment_param_23]

import time

start = time.time()


x_split, y_split, jet_num_to_idx = pp.preprocess_jet_num(x, y, to_replace, do_normalise=True, augment_param_list=augment_param_list)

end = time.time()
print(end - start)

787.7394399642944


In [6]:
def train_and_eval(data_part):

    models = []

    learning_param = {
            'lambda_': 1e-6,
            'gamma': 1e-6,
            'max_iters': 200
    }

    # lambdas = np.logspace(-4, 0, 30)
    # lambdas = [0, 1e-8, 1e-7, 1e-6, 1e-5, 1e-4]
    lambdas = [learning_param['lambda_']]

    acc_list = [[] for _ in lambdas]

    for i, lambda_ in enumerate(lambdas):
        learning_param['lambda_'] = lambda_

        for idx in range(len(data_part)):
            x_tr, y_tr = data_part[idx]['train']
            nb_features = x_tr.shape[1]

            w_initial = np.random.uniform(0,1,size=nb_features) 
            model_ls = model.Model('ridge_regression', w_initial, learning_param, debug=True)

            model_ls.learn_weights(y_tr, x_tr)
            models.append(model_ls)

            tx_te, y_te = data_part[idx]['test']
            y_ = model_ls.predict(tx_te)
            acc_list[i].append(misc.accuracy(y_te, y_))
            
    return acc_list, nb_samples

In [ ]:
cumul_eval = []

nb_runs = 10

for idx in range(nb_runs):
    print('{} out of {}', idx + 1, nb_runs)
    data_part = []

    nb_samples = []
    for x_, y_ in zip(x_split, y_split):
        train_data, test_data = misc.random_data_split(y_, x_, k_fold=10)
        data = {'train': train_data, 'test': test_data}
        data_part.append(data)
        nb_samples.append(len(test_data[1]))
        
    eval_result = train_and_eval(data_part)
    cumul_eval.append(eval_result)

In [23]:
avg_acc = np.dot(acc_list, nb_samples) / sum(nb_samples)
avg_acc

array([0.79416532])